In [78]:
import numpy as np
from scipy.stats import expon, chi2
import statistics as st
from math import exp
%matplotlib inline

LOC = 0
SCALE = 5
N = 100
L = 10

ɑ = 0.1

rng = np.random.default_rng(81381)
dist = LOC + rng.exponential(scale=SCALE, size=N)

# Критерий Χ^2 Пирсона

In [79]:
# def interval(hist, bin_centers):
# 	n = len(hist)
# 	k = n // 2

# 	for i in range(0, k):
# 		j = n - i - 1
		
# 		if hist[i] <= 5:
# 			bin_centers[i + 1] = bin_centers[i]
# 			hist[i + 1] += hist[i]
# 			hist[i] = 0
		
# 		if i == j:
# 			break

# 		if hist[j] <= 5:
# 			bin_centers[j - 1] = bin_centers[j]
# 			hist[j - 1] += hist[j]
# 			hist[j] = 0

# 	_k = 0
# 	_hist = []
# 	_bin_centers = []
# 	for i in range(0, k):
# 		if hist[i] > 0:
# 			_k += 1
# 			_hist.append(hist[i])
# 			_bin_centers.append(bin_centers[i])
	
# 	return (np.array(_hist), np.array(_bin_centers), _k)


def chi2_pirson():
	k = L
	hist, bin_edges = np.histogram(dist, bins=k)
	bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2

	mx = 1 / N * sum([hist[i] * bin_centers[i] for i in range(0, k)])
	Dx = 1 / (N - 1) * sum(
		[hist[i] * pow(bin_centers[i] - mx, 2) for i in range(0, k)]
	)
	σ = pow(Dx, 0.5)

	z = abs(bin_centers - mx) / σ
	_np = N * (bin_edges[1] - bin_edges[0]) / σ * expon.pdf(z)

	_chi2 = sum(pow(hist - _np, 2) / _np)

	return (_chi2, len(_np), mx, Dx, σ)


(X_obs, k, mx, Dx, σ) = chi2_pirson()
X_exp = chi2.ppf(1 - ɑ, k - 3)

if X_obs < X_exp:
	print('Гипотеза H0 принимается с заданным уровнем значимости')
else:
	print('Гипотеза H0 отвергается с заданным уровнем значимости')

Гипотеза H0 отвергается с заданным уровнем значимости


# Критерий Колмогорова

In [87]:
def kolm():
	x = np.sort(dist)
	mx = st.mean(x)
	Dx = st.variance(x)
	σ = st.stdev(x)

	p = []
	for i in range(0, N):
		p.append( expon.cdf(x[i], scale=mx) )
		x[i] = 1 / N - p[i]
		p[i] = p[i] - (i - 1) / N
	p = np.array(p)

	_x = np.stack([x, p])
	
	return _x.max() * pow(N, 0.5)

D_obs = kolm()

arg = []
x1 = []
for i in range(1, 20):
	arg.append(0.1 * i)
	x1.append(1 + 2 * sum([pow(-1, j) * exp(-2 * pow(j, 2) * pow(arg[i - 1], 2)) for j in range(1, 100)]))

D_kp = np.interp(1 - ɑ, x1, arg)

if D_obs < D_kp:
	print('Гипотеза H0 принимается с заданным уровнем значимости')
else:
	print('Гипотеза H0 отвергается с заданным уровнем значимости')

Гипотеза H0 принимается с заданным уровнем значимости
